In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/hamdan-codes/none-but-some/main/dataset.csv')

In [ ]:
df.head()

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document_type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,U001,200769623,WAL-MAR corp,11-02-2020 00:00,2020,1930438491,26-01-2020,20200125,20200126,20200210,USD,RV,1,NaN,54273.28,20200126,NAH4,1.930438e+09,0
1,U001,200980828,BEN E,08-08-2019 00:00,2019,1929646410,22-07-2019,20190722,20190722,20190811,USD,RV,1,NaN,79656.60,20190722,NAD1,1.929646e+09,0
2,U001,200792734,MDV/ trust,30-12-2019 00:00,2019,1929873765,14-09-2019,20190914,20190914,20190929,USD,RV,1,NaN,2253.86,20190914,NAA8,1.929874e+09,0
3,CA02,140105686,SYSC llc,NaN,2020,2960623488,30-03-2020,20200330,20200330,20200410,CAD,RV,1,NaN,3299.70,20200331,CA10,2.960623e+09,1
4,U001,200769623,WAL-MAR foundation,25-11-2019 00:00,2019,1930147974,13-11-2019,20191113,20191113,20191128,USD,RV,1,NaN,33133.29,20191113,NAH4,1.930148e+09,0


In [ ]:
df.corr()

,buisness_year,doc_id,document_create_date,document_create_date.1,due_in_date,posting_id,area_business,total_open_amount,baseline_create_date,invoice_id,isOpen
buisness_year,1.000000,-0.015241,0.979477,0.984438,0.990314,NaN,NaN,0.002743,0.984763,-0.013742,0.753522
doc_id,-0.015241,1.000000,-0.011879,-0.013437,-0.016548,NaN,NaN,0.168460,-0.011666,1.000000,-0.008764
document_create_date,0.979477,-0.011879,1.000000,0.994547,0.976467,NaN,NaN,0.006847,0.994078,-0.010089,0.762133
document_create_date.1,0.984438,-0.013437,0.994547,1.000000,0.980958,NaN,NaN,0.005598,0.999527,-0.011609,0.762191
due_in_date,0.990314,-0.016548,0.976467,0.980958,1.000000,NaN,NaN,0.003983,0.981420,-0.014778,0.754217
posting_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
area_business,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total_open_amount,0.002743,0.168460,0.006847,0.005598,0.003983,NaN,NaN,1.000000,0.005972,0.178028,0.013875
baseline_create_date,0.984763,-0.011666,0.994078,0.999527,0.981420,NaN,NaN,0.005972,1.000000,-0.009761,0.761918
invoice_id,-0.013742,1.000000,-0.010089,-0.011609,-0.014778,NaN,NaN,0.178028,-0.009761,1.000000,-0.007517


In [ ]:
train_val_data = df[df['clear_date'].notnull()]
test_data = df[df['clear_date'].isnull()] 

In [ ]:
le_business_code = preprocessing.LabelEncoder()
le_business_code.fit(df['business_code'].unique()) 

le_cust_number = preprocessing.LabelEncoder()
le_cust_number.fit(df['cust_number'].unique()) 

le_name_customer = preprocessing.LabelEncoder()
le_name_customer.fit(df['name_customer'].unique()) 

le_document_type = preprocessing.LabelEncoder()
le_document_type.fit(df['document_type'].unique()) 

le_cust_payment_terms = preprocessing.LabelEncoder()
le_cust_payment_terms.fit(df['cust_payment_terms'].unique()) 

LabelEncoder()

In [ ]:
def transform(df):
    # To transform object datatype into numeric data
    df['business_code'] = le_business_code.transform(df['business_code'])
    df['cust_number'] = le_cust_number.transform(df['cust_number'])
    df['name_customer'] = le_name_customer.transform(df['name_customer'])
    df['document_type'] = le_document_type.transform(df['document_type'])
    df['cust_payment_terms'] = le_cust_payment_terms.transform(df['cust_payment_terms'])
    

def preprocessToFeed(df):
    # Preprocess Data to feed to model
    rep = {'USD' : 1.0, 'CAD' : 0.79}
    df.replace(to_replace=rep, inplace=True)
    df['total_open_amount'] *= df['invoice_currency']
    
    df.drop(axis=1, 
            columns=[
                'area_business', 'posting_id', 'invoice_id', 'invoice_currency', 'document_create_date', 
                'document_create_date.1', 'baseline_create_date', 'buisness_year', 'posting_date', 
                'isOpen'
            ], 
            inplace=True
        )
    
    df['clear_date'] = pd.to_datetime(df['clear_date'], format='%d-%m-%Y 00:00')
    
    df['due_in_date'] = df.due_in_date.astype('int64')
    df['due_in_date'] = pd.to_datetime(df['due_in_date'], format='%Y%m%d')
    
    transform(df)
    
    
def preprocessTestData(df):
    # To preprocess Test Data to feed directly to model prediction
    preprocessToFeed(df)
    df.drop(columns=['clear_date'], inplace=True)
     

In [ ]:
train_val_data.head()

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document_type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen
0,U001,200769623,WAL-MAR corp,11-02-2020 00:00,2020,1930438491,26-01-2020,20200125,20200126,20200210,USD,RV,1,NaN,54273.28,20200126,NAH4,1.930438e+09,0
1,U001,200980828,BEN E,08-08-2019 00:00,2019,1929646410,22-07-2019,20190722,20190722,20190811,USD,RV,1,NaN,79656.60,20190722,NAD1,1.929646e+09,0
2,U001,200792734,MDV/ trust,30-12-2019 00:00,2019,1929873765,14-09-2019,20190914,20190914,20190929,USD,RV,1,NaN,2253.86,20190914,NAA8,1.929874e+09,0
4,U001,200769623,WAL-MAR foundation,25-11-2019 00:00,2019,1930147974,13-11-2019,20191113,20191113,20191128,USD,RV,1,NaN,33133.29,20191113,NAH4,1.930148e+09,0
5,CA02,140106181,THE corporation,04-12-2019 00:00,2019,2960581231,20-09-2019,20190920,20190920,20191004,CAD,RV,1,NaN,22225.84,20190924,CA10,2.960581e+09,0


In [ ]:
preprocessToFeed(train_val_data)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/p

In [ ]:
train_val_data.head()

,business_code,cust_number,name_customer,clear_date,doc_id,due_in_date,document_type,total_open_amount,cust_payment_terms
0,1,908,4065,2020-02-11,1930438491,2020-02-10,0,54273.2800,39
1,1,1072,385,2019-08-08,1929646410,2019-08-11,0,79656.6000,33
2,1,977,2537,2019-12-30,1929873765,2019-09-29,0,2253.8600,22
4,1,908,4067,2019-11-25,1930147974,2019-11-28,0,33133.2900,39
5,0,447,3805,2019-12-04,2960581231,2019-10-04,0,17558.4136,6


In [ ]:
train_val_data['delay'] = (train_val_data.clear_date - train_val_data.due_in_date)
train_val_data.drop(columns=['clear_date'], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train_val_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   business_code       40000 non-null  int64          
 1   cust_number         40000 non-null  int64          
 2   name_customer       40000 non-null  int64          
 3   doc_id              40000 non-null  int64          
 4   due_in_date         40000 non-null  datetime64[ns] 
 5   document_type       40000 non-null  int64          
 6   total_open_amount   40000 non-null  float64        
 7   cust_payment_terms  40000 non-null  int64          
 8   delay               40000 non-null  timedelta64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6), timedelta64[ns](1)
memory usage: 3.1 MB


In [ ]:
train_val_data['delay'] = (train_val_data['delay'] / np.timedelta64(1,'D')).astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train_val_data.corr()

,business_code,cust_number,name_customer,doc_id,document_type,total_open_amount,cust_payment_terms,delay
business_code,1.000000,0.130647,-0.026749,-0.496002,-0.022394,-0.073141,0.488137,-0.008252
cust_number,0.130647,1.000000,0.208885,-0.485951,-0.021095,-0.043809,0.362026,0.011982
name_customer,-0.026749,0.208885,1.000000,-0.016507,0.007159,-0.018840,0.136280,-0.024530
doc_id,-0.496002,-0.485951,-0.016507,1.000000,0.312764,0.091678,-0.479525,0.164581
document_type,-0.022394,-0.021095,0.007159,0.312764,1.000000,-0.009079,-0.021503,0.079528
total_open_amount,-0.073141,-0.043809,-0.018840,0.091678,-0.009079,1.000000,-0.170683,-0.001307
cust_payment_terms,0.488137,0.362026,0.136280,-0.479525,-0.021503,-0.170683,1.000000,0.029239
delay,-0.008252,0.011982,-0.024530,0.164581,0.079528,-0.001307,0.029239,1.000000


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    train_val_data.drop(columns=['due_in_date', 'delay']),
    train_val_data['delay'], 
    test_size=0.2, 
    random_state=0
)

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((32000, 7), (32000,), (8000, 7), (8000,))

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)
y_val_pred = model.predict(x_val)

y_val_pred = y_val_pred + 0.5
y_val_pred = y_val_pred.astype('int')

x = y_val - y_val_pred
error = (x.dot(x) / len(x)) ** 0.5

print('Error (Root mean squared error):', error)

Error (Root mean squared error): 10.24296099768031


In [ ]:
def predToOutput(y):
    # Function to convert predicted output in desired format
    y = y + 0.5
    y = y.astype('int')
    y = pd.to_timedelta(y, unit='D') + test_data['due_in_date']
    y = y.dt.strftime("%d-%m-%Y 00:00")
    
    return y

In [ ]:
preprocessTestData(test_data)
# Preprocess and feed data to predict
y_pred = model.predict(test_data.drop(columns=['due_in_date']))
y_pred = predToOutput(y_pred)
y_pred

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/p

3        16-04-2020 00:00
7        03-04-2020 00:00
10       26-03-2020 00:00
14       30-04-2020 00:00
15       26-04-2020 00:00
               ...       
49975    26-03-2020 00:00
49980    19-05-2020 00:00
49982    25-03-2020 00:00
49992    09-04-2020 00:00
49995    06-05-2020 00:00
Length: 10000, dtype: object

In [ ]:
print('Output:')
print(y_pred)

Output:
3        16-04-2020 00:00
7        03-04-2020 00:00
10       26-03-2020 00:00
14       30-04-2020 00:00
15       26-04-2020 00:00
               ...       
49975    26-03-2020 00:00
49980    19-05-2020 00:00
49982    25-03-2020 00:00
49992    09-04-2020 00:00
49995    06-05-2020 00:00
Length: 10000, dtype: object
